In [98]:
import requests
import plaid
from plaid.model.link_token_create_request import LinkTokenCreateRequest
from plaid.model.link_token_create_request_user import LinkTokenCreateRequestUser
from plaid.model.products import Products
from plaid.model.country_code import CountryCode
from plaid.api import plaid_api
host = plaid.Environment.Sandbox

# Replace with your Plaid API credentials
PLAID_CLIENT_ID = '6574857728575f001c67f5d9'
PLAID_SECRET = '7ecc18ddda654fdf2971518e33c19c'

# Plaid API endpoint for creating a link token
PLAID_API_URL = 'https://sandbox.plaid.com/link/token/create'

# Parameters for the API request
# Account filtering isn't required here, but sometimes 
# it's helpful to see an example. 
configuration = plaid.Configuration(
    host=host,
    api_key={
        'clientId': PLAID_CLIENT_ID,
        'secret': PLAID_SECRET
    }
)

api_client = plaid.ApiClient(configuration)
client = plaid_api.PlaidApi(api_client)

request = LinkTokenCreateRequest(
  user=LinkTokenCreateRequestUser(
    client_user_id='user-id',
  ),
  client_name='BudgetBuddy',
  products=[Products('transactions')],
  country_codes=[CountryCode('US')],
  language='en',
  )

# create link token
# response = client.link_token_create(request)
# link_token = response['link_token']
# params = {
    
#     'client_id': PLAID_CLIENT_ID,
#     'secret': PLAID_SECRET,

#     'user': {
#         'client_user_id': '6574857728575f001c67f5d9',  # Replace with your user ID
#     },
#     'products': ['transactions'],
# }

# Make the API request to obtain the link token
# response = requests.post(PLAID_API_URL, json=params)
try:
    response = client.link_token_create(request)
    data = response.to_dict()  # Assuming Plaid's response has a to_dict method
    link_token = data.get('link_token')
# Handle the response
    if link_token:
        print(f"Link Token: {link_token}")
    else:
        print(f"Error: Link Token not present in the response data")
except plaid.ApiException as e:
    error_data = json.loads(e.body)
    print(f"Error: {error_data['error_code']} - {error_data['error_message']}")
link_token = 'link-sandbox-944b097f-fb64-40a7-9d1d-6a2ee719ba71'

Link Token: link-sandbox-d73f049d-f5af-44d1-b43e-88656d489d2d


In [99]:
from plaid.model.sandbox_public_token_create_request import SandboxPublicTokenCreateRequest
pt_request = SandboxPublicTokenCreateRequest(
    institution_id='ins_109508',
    initial_products=[Products('transactions')]
)
pt_response = client.sandbox_public_token_create(pt_request)
# The generated public_token can now be
# exchanged for an access_token
exchange_request = ItemPublicTokenExchangeRequest(
    public_token=pt_response['public_token']
)
exchange_response = client.item_public_token_exchange(exchange_request)

In [100]:
# Provide a cursor from your database if you've previously
# received one for the Item. Leave null if this is your
# first sync call for this Item. The first request will
# return a cursor.
transactions_db = []
latest_cursor_db = {}
def get_latest_cursor_or_none(item_id):
    return latest_cursor_db.get(item_id, '')

def apply_updates(item_id, added, modified, removed, cursor):
    global transactions_db
    transactions_db.extend(added)
    transactions_db = [t for t in transactions_db if t not in removed]
    
    # Update the latest cursor for the item
    latest_cursor_db[item_id] = cursor

item_id = 'BzKjAw6MMpHqJz445wGgHa8rXeKGvQtwkWl3M'
latest_cursor = get_latest_cursor_or_none(item_id)
# New transaction updates since "cursor"
added = []
modified = []
removed = [] # Removed transaction ids
has_more = True

# Iterate through each page of new transaction updates for item
while has_more:
  request = TransactionsSyncRequest(
    access_token=access_token,
    cursor=latest_cursor,
  )
  response = client.transactions_sync(request)

  # Add this page of results
  added.extend(response['added'])
  modified.extend(response['modified'])
  removed.extend(response['removed'])

  has_more = response['has_more']

  # Update cursor to the next cursor
  latest_cursor = response['next_cursor']

# Persist cursor and updated data
apply_updates(item_id, added, modified, removed, latest_cursor)

In [101]:
import pandas as pd
df = pd.DataFrame(transactions_db)
df

,0
0,{'account_id': 'K5BXk6PMMxHkwQBdnpq7IpG6o3km3K...
1,{'account_id': 'K5BXk6PMMxHkwQBdnpq7IpG6o3km3K...
2,{'account_id': 'K5BXk6PMMxHkwQBdnpq7IpG6o3km3K...
3,{'account_id': 'K5BXk6PMMxHkwQBdnpq7IpG6o3km3K...
4,{'account_id': 'K5BXk6PMMxHkwQBdnpq7IpG6o3km3K...
5,{'account_id': 'K5BXk6PMMxHkwQBdnpq7IpG6o3km3K...
6,{'account_id': 'K5BXk6PMMxHkwQBdnpq7IpG6o3km3K...
7,{'account_id': 'rdQPE1weeBH9JNx5qGZXSrlQxnz6nm...
8,{'account_id': 'rdQPE1weeBH9JNx5qGZXSrlQxnz6nm...
9,{'account_id': 'zQ8LKVzggjuDlpWAoMyghQ89qxjkxB...
